# Model Fitting


One of the most common things in scientific computing is model fitting. Numerical Recipes devotes a number of chapters to this.

* scipy "curve_fit"
* astropy
* lmfit (emcee)
* pyspeckit

We will also need to be able to read data. We will use pickle and an ascii file reader.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab


# pickle
you may need to re-visit n6503-case1 and execute **In[20]** to get the file

In [ ]:
!ls -l n6503-sp.p


In [ ]:
try:
    import cPickle as pickle
except:
    import pickle
   
sp = pickle.load(open("n6503-sp.p","rb"))
print(sp.keys())
velz = sp['z']
flux = sp['i']
plt.plot(velz,flux)
plt.xlabel(sp['zunit'])
plt.ylabel(sp['iunit'])
plt.title("Pos: %s %s" % (str(sp['xpos']),str(sp['ypos'])))
#  plt.title("Pos: %d %d" % (sp['xpos'],sp['ypos']))

In [ ]:
# compute moments of this spectrum to get an idea what the mean and dispersion of this signal is
# recall this method can easily result in zdisp < 0 for noisy data
tmp = flux*velz 
tmp2 = flux*velz*velz
zmean =tmp.sum()/flux.sum()
zdisp = tmp2.sum()/flux.sum() - zmean*zmean

print(zmean,zdisp)

The [scipy](https://docs.scipy.org/doc/scipy/reference/) module has a large number of optimization and fitting routines that work with numpy arrays. They directly call lower level C routines, generally making fitting a fast process.

To fit an actual gaussian , instead of using moments, we use the [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit)
function in [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html):


In [ ]:
from scipy.optimize import curve_fit

def gauss(x, *p):
    # if len(p) != 3: raise ValueError("Error, found %d, (%s), need 3" % (len(p),str(p)))
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

# p0 is the initial guess for the fitting coefficients (A, mu and sigma above, in that order)
p0 = [1, 10., 10.]
p0 = [0.1, 10., 10.]
p0 = [0.01, 10., 10.]

coeff, var_matrix = curve_fit(gauss, velz, flux, p0=p0)
flux_fit = gauss(velz, *coeff)

plt.plot(velz,flux,     label='Data')
plt.plot(velz,flux_fit, label='Fit')
plt.legend()

print("Fitted amp",coeff[0])
print("Fitted mean",coeff[1])
print("Fitted sigma",coeff[2])

# PySpecKit

PySpecKit is an extensible spectroscopic analysis toolkit for astronomy. See
http://pyspeckit.bitbucket.org/html/sphinx/index.html

Installing this with
```
    pip install pyspeckit
```
resulted in an error 
```
    AttributeError: module 'distutils.config' has no attribute 'ConfigParser'
```
turns out this was a python2/3 hack that was needed. The current released version of pyspeckit did not handle this. A manual install of the development release solved this:
```
    pip install https://bitbucket.org/pyspeckit/pyspeckit/get/master.tar.gz
```


### modify to run

The cell below is an adapted version of a gaussian fit case from the pySpecKit manual.  Change it to make it work with our spectrum from n6503.  How does it compare to scipy's curve_fit ?

In [ ]:
import pyspeckit

# set up a gauss (amp,center,sigma)
xaxis = np.linspace(-50.0,150.0,100)
amp = 1.0
sigma = 10.0
center = 50.0
synth_data = amp*np.exp(-(xaxis-center)**2/(sigma**2 * 2.))

# Add 10% noise (but fix the random seed)
np.random.seed(123)
stddev = 0.1
noise = np.random.randn(xaxis.size)*stddev
error = stddev*np.ones_like(synth_data)
data = noise+synth_data

# this will give a "blank header" warning, which is fine
sp = pyspeckit.Spectrum(data=data, error=error, xarr=xaxis,
                        xarrkwargs={'unit':'km/s'},
                        unit='erg/s/cm^2/AA')

sp.plotter()

# Fit with automatic guesses
sp.specfit(fittype='gaussian')

# Fit with input guesses
# The guesses initialize the fitter
# This approach uses the 0th, 1st, and 2nd moments
amplitude_guess = data.max()
center_guess = (data*xaxis).sum()/data.sum()
width_guess = data.sum() / amplitude_guess / np.sqrt(2*np.pi)
guesses = [amplitude_guess, center_guess, width_guess]
sp.specfit(fittype='gaussian', guesses=guesses)

sp.plotter(errstyle='fill')
sp.specfit.plot_fit()